### Práctica 1 - Predicción Energía Eólica
Autores:
- Miguel Domínguez Gómez - 100451258
- Eduardo Fernández Vega - 100472251  

Enlace al repositorio:
https://github.com/MiguelMHR/Practica1-PrediccionEnergia


### 0. Tabla de contenidos:
1. [Requisitos](#1.-Requisitos)
2. [Lectura](#2.-Lectura)
3. [EDA](#3.-EDA)
4. [Evaluación inner y outer](#4.-Evaluación-inner-y-outer) 
5. [Elección del mejor escalador](#5.-Elección-del-mejor-escalador)
6. [Evaluación de los modelos](#6.-Evaluación-de-los-modelos)
7. [Creación modelo final](#7.-Creación-modelo-final)
8. [Transformación problema regresión a clasificación](#8.-Transformación-problema-regresión-a-clasificacion)
9. [Uso de AI en nuestro proyecto](#9.-Uso-de-AI-en-nuestro-proyecto)

### 1. Requisitos

Importamos las librerías necesarias para el proyecto

In [ ]:
# Importando las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, KFold, TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR, SVC
from time import time
import pickle


### 2. Lectura

Procedemos a leer el dataset usando la librería Pandas

In [ ]:
# Leemos el dataset
wind_ava = pd.read_csv('./data/wind_ava.csv')

### 3. EDA

En esta sección se hará un análisis exploratorio de datos siguiento las cuestiones expuestas en el enunciado de la práctica

#### 3.0 Preparación previa del dataset
Como se expone en el punto **importante** de la práctica, debemos, antes de realizar el EDA simplificado, una limpieza de las columnas que no sean importantes. Las únicas que son relevantes son aquellas que tienen que ver con Sotavento:
- **datetime**
- **energy**
- solo aquellas que tengan un ".13" al final (**p54.162.13, p55.162.13, cape.13, etc**)

In [ ]:
# Para quedarme solo con las columnas que terminan por ".13" o que sean "energy" o "datetime"
columnas_a_mantener = [columna for columna in wind_ava.columns if columna.endswith('.13') or columna == 'energy' or columna == 'datetime']
wind_ava = wind_ava[columnas_a_mantener]
print(wind_ava.head(1))


#### 3.1 Descripción general del dataset
Para poder conocer ciertas características relevantes del dataset, como el número de instancias (filas) y característas (columnas) procederemos a usar diferentes funciones de Pandas con el dataset transformado

In [ ]:
# Descripción de parámetros generales -> count, mean, std, min, 25%, 50%, 75%, max
print(wind_ava.describe())

# Número de filas y columnas del dataset
print("\n")
print("Número de filas: ", wind_ava.shape[0])
print("Número de columnas: ", wind_ava.shape[1])
print("\n")

# Para saber el tipo de variable de cada columna
print(wind_ava.info())

Como se puede apreciar, no existen valores nulos ya que el número de filas (4748) coincide con la cuenta de los valores no-nulos de cada columna

#### 3.2 Valores faltantes

Como ya se ha visto en el anterior apartado, no existen valores nulos como tal. Para asegurarnos, haremos lo siguiente

In [ ]:
# Para poder saber el número de valores faltantes:
print(wind_ava.isnull().sum())


Ahora, comprobaremos, como todos son números decimales, si por casualidad hay algún cero que no tengan sentido en el dataset

In [ ]:
# Ahora, comprobaremos, como todos son números decimales, si por casualidad hay algún cero que no tengan sentido en el dataset
# Para ello, veremos el número de ceros que hay en cada columna
print((wind_ava == 0).sum())

Se puede apreciar que en cape.13 hay bastantes valores que son cero pero, como esa variable tiene sentido que sea cero en determinados contextos, como en condiciones atmosféricas estables, se ha decidido mantener así

#### 3.3 Columnas constantes
En este apartado se evaluará qué columnas son constantes para todos sus valores

In [ ]:
# Encontrar las columnas que tienen un solo valor único (columnas constantes)
constant_columns = wind_ava.columns[wind_ava.nunique() <= 1].tolist()
print(constant_columns)


Como se puede apreciar, no existen columnas constantes, por lo que no se va a eliminar nada del dataset

#### 3.4 ¿Regresión o clasificación?
Directamente conociendo la naturaleza de la variable objetivo, la energía, se puede saber ante qué tipo de problema nos enfrentamos. Como la energía es una variable continua, nos encontramos ante un problema de **regresión**. 


#### 3.5 Otras consideraciones: outliers
Otro apartado del EDA que no se menciona en el enunciado y que creemos que es de vital importancia para realizar un buen análisis exploratorio es evaluar los valores lejanos, o outliers

#### 3.5.1 Outliers con Histograma

Usaremos varios métodos de representación de los datos para poder observar los valores alejados

In [ ]:

# Quitamos 'datetime' al no ser una columna numérica
columna_excluida = 'datetime'
plt.hist(wind_ava[wind_ava.columns.difference([columna_excluida])])
plt.show()

Se puede apreciar que hay ciertos valores que se salen de la norma, aunque se verá más claro realizando los boxplots

##### 3.5.2 Outliers con Boxplot
Para poder evaluar atributo a atributo, se procederá a hacer un boxplot de los 21 atributos (menos datetime, al no ser numérico y no tener outliers al ser un registro de las mediciones diarias)

In [ ]:
# Boxplot para cada atributo y sacamos los outliers de cada uno
outliers = []
for attribute in wind_ava.columns.difference([columna_excluida]):
    plt.boxplot(wind_ava[attribute])
    plt.title(attribute)
    plt.show()
    Q1 = wind_ava[attribute].quantile(0.25)
    Q3 = wind_ava[attribute].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers.append(wind_ava[(wind_ava[attribute] < lower_bound) | (wind_ava[attribute] > upper_bound)].index)

Como muestran los resultados, hay ciertos atributos que poseen bastantes outliers, otros que poseen algunos y hay otros atributos en los que es inexistente. Hace falta determinar si esos son ruido, para ser eliminados en futuros modelos para mejorar la predicción, o si son datos realmente significativos para el dataset. Al ser variables meteorológicas, cabe la posibilidad de que haya cierto fallo en la medición, pero al ser la metereología de una región algo tan cambiante, se cree que esa información debe prevalecer en el dataset, a pesar de poseer outliers

### 4. Evaluación inner y outer

En este apartado se va a describir el proceso seguido para realizar tanto la evaluación inner como la outer:

Se ha decidido dividir el dataset en **train** y **test** de la siguiente forma, siguiendo el criterio, no exactamente, de *2/3 para train* y *1/3* *para* *test*:

* *train:* 2005 al 2007 (**60%**)
* *test:* 2008 y 2009 (**40%**)

No se ha decidido añadir aleatoriedad a la selección de instancias a las particiones *train* y *test* ya que hay dependencia temporal en los datos. Es decir, no se pueden mezclar datos del 2005 con datos del 2009 ya que podrían afectar al aprendizaje del modelo. 

No se ha decidido seguir al 100% *2/3 para train* y *1/3* *para test* al querer dividir las particiones por años exactos.



Por otro lado, dentro del *train*, se va a realizar un **ajuste de hiperparámetros** para la evaluacion inner, dividiendo *train*, siguiendo TimeSeriesSplit, con 2 splits, de la siguiente forma:

* **Prueba #1**:
  * *train_train:* 2005
  * *train_validation: 2006*
* **Prueba #2:**
  * *train_train:* 2005 y 2006
  * *train_validation:* 2007

Se ha decidido usar TimeSeriesSplit para dividir *train* por la misma razón por la que no añadiamos aleatoriedad a la selección de instancias. Al trabajar con datos con dependencia temporal, es mejor usar TimeSeriesSplit. Además, se ha decidido usar GridSearch al ser el método de ajuste de hiper-parámetros óptimo (computacionalmente es costoso, pero da los mejores resultados)

Los hiperparámetros que se van a ajustar dependen del modelo a usar y se comentarán en el punto 4


Para la evaluación outer, se evaluará el rendimiento usando la partición *train* inicial y la partición *test* de 2008 y 2009, es decir, el dataset completo, usando los mejores hiperparámetros encontrados en la evaluación inner. 


Las **métricas** que se van a usar para evaluar el rendimiento a futuro van a ser:

* **RMSE (Root mean squared error) :** muy recomendado como métrica de error
* **Tiempo de ejecución :** es una variable relevante para poder ofrecer un modelo que pueda ejecutar en un tiempo razonable. También, se usará como medio de desempate si hay modelos que tienen errores muy parecidos.

Aunque siempre se priorizará el modelo que tenga menos error, habrá una excepción. Si el tiempo de ejecución es infinitamente superior en uno de los dos modelos a comparar y poseen errores muy parecidos, se elegirá el que tenga menor tiempo de ejecución.


In [ ]:
# ----  Selección inicial de X e Y  ----
# X -> Todas las columnas menos 'datetime' y 'energy'
# Y -> 'energy'
X = wind_ava.loc[:,columnas_a_mantener[2:]]
Y = wind_ava.energy

# ----  División del dataset en Train-Test:  ----
# Dividimos el dataset en entrenamiento del 2005 al 2007 y test del 2007 al 2009
X_train = X.loc[wind_ava.datetime < '2008-01-01']
X_test = X.loc[wind_ava.datetime >= '2008-01-01']
Y_train = Y.loc[wind_ava.datetime < '2008-01-01']
Y_test = Y.loc[wind_ava.datetime >= '2008-01-01']

# ----  TimeSeriesSplit  ----
tscv = TimeSeriesSplit(n_splits=2)


### 5. Elección del mejor escalador
Para elegir el mejor escalador, se hará varias pruebas usando KNN por omisión con 3-Fold cross validation, una por cada scaler:
MinMax, Standard y Robust

In [ ]:
"""
Si el método a usar posee random_state, usar 100451258 (mi NIA)
"""

# Establecemos la semilla aleatoria
np.random.seed(100451258)

# ----  Evaluación inner con 3-Fold CV  ----
inner = KFold(n_splits=3, shuffle=True)
# Diccionario para almacenar los resultados de la evaluación inner  ----
inner_scores = {}

# ----  1. KNN con StandardScaler  ----
# Creamos el pipeline
pipeline_std = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsRegressor())])
# Evaluación inner
scores_std = cross_val_score(pipeline_std, X_train, Y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['KNN con StandardScaler'] = -scores_std.mean()

# ----  2. KNN con MinMaxScaler  ----
# Creamos el pipeline
pipeline_mm = Pipeline([('scaler', MinMaxScaler()), ('knn', KNeighborsRegressor())])
# Evaluación inner
scores_mm = cross_val_score(pipeline_mm, X_train, Y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['KNN con MinMaxScaler'] = -scores_mm.mean()

# ----  3. KNN con RobustScaler  ----
# Creamos el pipeline
pipeline_rb = Pipeline([('scaler', RobustScaler()), ('knn', KNeighborsRegressor())])
# Evaluación inner
scores_rb = cross_val_score(pipeline_rb, X_train, Y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['KNN con RobustScaler'] = -scores_rb.mean()

# ----  RESULTADOS FINALES  ----
for name, score in inner_scores.items():
    print(f"{name}: {score}")


Como se puede apreciar, el mejor scaler y el que usaremos de ahora en adelante es StandardScaler

### 6. Evaluación de los modelos
Se procederá a realizar la evaluación de los siguientes modelos.
- **KNN**:
  - Por omisión de hiperparámetros
  - Por ajuste de hiperparámetros
- **Árboles de decisión**
  - Por omisión de hiperparámetros
  - Por ajuste de hiperparámetros
- **Regresión lineal**
  - Normal : por omisión de hiperparámetros
  - Lasso : por ajuste de hiperparámetros
- **SVM**
  - Por omisión de hiperparámetros
  - Por ajuste de hiperparámetros
- **Regresión Dummy**
  - Por omisión de hiperparámetros

Una vez se terminen las evaluaciones, se describirán las conclusiones alcanzadas en este apartado

Antes de comenzar, debemos escalar los datos usando el **StandardScaler**, el mejor escalador hayado en el punto anterior

In [ ]:
# ----  Estandarizacion de los datos:  ----
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### 6.1 KNN

##### 6.1.1 KNN por omisión de hiperparámetros

In [ ]:
np.random.seed(100451258)

# ----  1. KNN sin hiperparámetros  ----
knn = KNeighborsRegressor()
start = time()
scores = cross_val_score(knn, X, Y, scoring='neg_root_mean_squared_error', cv=tscv) 
end = time()

print(f"MSRE: {scores.mean():.2f} ± {scores.std():.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")


##### 6.1.2 KNN usando hiperparámetros
Los hiperparámetros elegidos para KNN son los siguientes:
- **n_neighbours** : número de vecinos
- **weights** : método de ponderación
  - uniform : todos los vecinos tienen el mismo peso
  - distance : los vecinos más cercanos tienen más peso
- **metric** : métrica de distancia de la cercanía de los puntos
  - euclidean : distancia euclídea
  - manhattan : distancia manhattan
  - minkowski : distancia minkowski


In [ ]:
np.random.seed(100451258)

# 0. Elección de hiperparámetros
knn_param = {
    'n_neighbors': range(1, 30),
    'weights': ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan', 'minkowski'],
}

# 1. Evaluación inner
knn = KNeighborsRegressor()

gs_regr = GridSearchCV(knn, 
                       knn_param, 
                       scoring='neg_root_mean_squared_error', 
                       cv=tscv, 
                       n_jobs=-1, 
                       )
# 2. Entrenamiento con la partición de entrenamiento
start = time()
gs_regr.fit(X_train, Y_train)
end = time()
best_hyperparams = gs_regr.best_params_
rmse = -gs_regr.best_score_
std = gs_regr.cv_results_['std_test_score'][gs_regr.best_index_]

print(f"Mejores hiperparámetros: {best_hyperparams}")
print(f"MSRE: {rmse:.2f} ± {std:.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

#### 6.2 Árboles de decisión

##### 6.2.1 Árboles de decisión por omisión de hiperparámetros

In [ ]:
np.random.seed(100451258)

# ----  1. Árboles de decisión sin hiperparámetros  ----
tree = DecisionTreeRegressor()
start = time()
scores = cross_val_score(tree, X, Y, scoring='neg_root_mean_squared_error', cv=tscv) 
end = time()

print(f"MSRE: {-scores.mean():.2f} ± {scores.std():.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

##### 6.2.2 Árboles de decisión por ajuste de hiperparámetros

Los hiperparámetros que han sido elegidos son los siguientes:
- **max_depth** : profundidad máxima
- **min_samples_split** : número mínimo de instancias que debe de
tener un nodo interno para ser subdividido
- **min_samples_leaf** : número mínimo de instancias requeridas para que un nodo sea considerado como una hoja

In [ ]:
np.random.seed(100451258)

# 0. Elección de hiperparámetros
tree_param = {
    'max_depth': range(2,16,2),
    'min_samples_split': range(2,16,2),
    'min_samples_leaf': range(2,16,2)
}

# 1. Evaluación inner
tree = DecisionTreeRegressor()

gs_regr = GridSearchCV(tree, 
                       tree_param, 
                       scoring='neg_root_mean_squared_error', 
                       cv=tscv, 
                       n_jobs=-1, 
                       )
# 2. Entrenamiento con la partición de entrenamiento
start = time()
gs_regr.fit(X_train, Y_train)
end = time()
best_hyperparams = gs_regr.best_params_
rmse = -gs_regr.best_score_
std = gs_regr.cv_results_['std_test_score'][gs_regr.best_index_]

print(f"Mejores hiperparámetros: {best_hyperparams}")
print(f"MSRE: {rmse:.2f} ± {std:.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

#### 6.3 Regresión lineal

##### 6.3.1 Regresión lineal normal (solo omisión)

In [ ]:
np.random.seed(100451258)

# ----  1. Árboles de decisión sin hiperparámetros  ----
reg_lin = LinearRegression()
start = time()
scores = cross_val_score(reg_lin, X, Y, scoring='neg_root_mean_squared_error', cv=tscv) 
end = time()
print(f"MSRE: {-scores.mean():.2f} ± {scores.std():.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

##### 6.3.2 Regresión Lasso (ajuste de hiperparámetros)

Se ha decidido ajustar el siguiente hiperparámetro:
- **alpha** : controla la fuerza de la regularización en el modelo Lasso

In [ ]:
np.random.seed(100451258)

# 0. Elección de hiperparámetros de lasso
lasso_param = {
    "alpha": np.logspace(-2, 5, 75)
}

# 1. Evaluación inner
lasso = Lasso()

gs_regr = GridSearchCV(lasso, 
                       lasso_param, 
                       scoring='neg_root_mean_squared_error', 
                       cv=tscv, 
                       n_jobs=-1, 
                       )
# 2. Entrenamiento con la partición de entrenamiento
start = time()
gs_regr.fit(X_train, Y_train)
end = time()
best_hyperparams = gs_regr.best_params_
rmse = -gs_regr.best_score_
std = gs_regr.cv_results_['std_test_score'][gs_regr.best_index_]

print(f"Mejores hiperparámetros: {best_hyperparams}")
print(f"MSRE: {rmse:.2f} ± {std:.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

#### 6.4 Máquina de soporte de vectores (SVM)

##### 6.4.1 SVM por omisión de hiperparámetros

In [ ]:
np.random.seed(100451258)

# ----  1. SVM sin hiperparámetros  ----
svm = SVR()
start = time()
scores = cross_val_score(svm, X, Y, scoring='neg_root_mean_squared_error', cv=tscv) 
end = time()

print(f"MSRE: {-scores.mean():.2f} ± {scores.std():.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")


##### 6.4.2 SVM con ajuste de hiperparámetros

Los hiperparámetros elegidos para ser ajustados son:
- **C** : controla el equilibrio entre la maximización del margen y la minimización de la clasificación errónea
- **epsilon** : controla la influencia de cada ejemplo de entrenamiento en el modelo
- **kernel** : funciones de kernel para transformar los datos en un espacio de mayor dimensión donde sea más fácil encontrar un hiperplano de separación lineal

In [ ]:
np.random.seed(100451258)

# 0. Elección de hiperparámetros
svm_param = {
    'C': [0.1, 1, 10, 100, 1000, 10000],
    'epsilon': [0.1, 0.2, 0.3, 0.4],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# 1. Evaluación inner
svm = SVR()

gs_regr = GridSearchCV(svm, 
                       svm_param, 
                       scoring='neg_root_mean_squared_error', 
                       cv=tscv, 
                       n_jobs=-1, 
                       )
# 2. Entrenamiento con la partición de entrenamiento
start = time()
gs_regr.fit(X_train, Y_train)
end = time()
best_hyperparams = gs_regr.best_params_
rmse = -gs_regr.best_score_
std = gs_regr.cv_results_['std_test_score'][gs_regr.best_index_]

print(f"Mejores hiperparámetros: {best_hyperparams}")
print(f"MSRE: {rmse:.2f} ± {std:.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

#### 6.5 Regresor Dummy (solo por omision)

In [ ]:
np.random.seed(100451258)

# ----  1. SVM sin hiperparámetros  ----
dummy = DummyRegressor()
start = time()
scores = cross_val_score(dummy, X, Y, scoring='neg_root_mean_squared_error', cv=tscv) 
end = time()

print(f"MSRE: {-scores.mean():.2f} ± {scores.std():.2f}")
print("Tiempo de ejecución: " + str(end-start) + " segundos")

#### 6.6 Conclusiones finales:
Al evaluar todos los modelos siguiendo todo el proceso descrito en el apartado 2, se han logrado obtener varias conclusiones:
- La utilización de hiperparámetros mejora considerablemente las predicciones, resultando en errores más bajos
  - Además, al tener que probar con varios modelos mientras se realiza el ajuste, tanto el coste computacional como el tiempo de ejecución aumentan. Como la mayor importancia reside en un error menor de los modelos, este hecho no nos preocupa ya que los tiempos, igualmente, son relativamente bajos y permisibles.
- El **mejor método básico** (sin ajuste de hiper parámetros) ha sido **árboles de decisión**
- El **mejor método** global (tanto con ajuste como sin ajuste) obtenido ha sido **SVM con ajuste de hiperparámetros**. Ha sido el que más ha tardado en ejecutar, pero el que mejor resultado ha brindado
- Por otro lado, los regresores dummy no han proporcionado mejores resultados que los métodos comentados anteriormente
- Finalmente, existe una proporcionalidad visible entre el tiempo de ejecución y el error obtenido. A mayor tiempo de ejecución, menor error obtenido

### 7. Construcción modelo final

Finalmente, una vez hemos obtenido el mejor modelo, SVM con hiperparámetros ('C': 1000, 'epsilon': 0.4, 'kernel': 'rbf'), se procederá a entrenar de nuevo el modelo usando los hiperparámetros obtenidos y **todos los datos**, es decir, teniendo en cuenta x_test. Además, se creará el modelo que hará las predicciones en otro ipynb

In [ ]:
np.random.seed(100451258)

# 0. Uso de hiperparámetros obtenidos
svm_param = {
    'C': 1000,
    'epsilon': 0.4,
    'kernel': 'rbf'
}

# 1. Entrenamiento con hiperparámetros de evaluación inner
svm = SVR(**svm_param)

# 2. Entrenamiento con la partición de entrenamiento
svm.fit(X_train, Y_train)

# 3. Evaluación outer del modelo
Y_pred = svm.predict(X_test)
mse = metrics.mean_squared_error(Y_test, Y_pred)
print("RMSE: ", np.sqrt(mse))

# 4. Construimos el modelo final con todos los datos sin hacer ajuste por hiperparámetros
# Computacionalmente más asequible y sigue siendo válido
svm.fit(X, Y)
mse_total = metrics.mean_squared_error(Y, svm.predict(X))
print("RMSE total: ", np.sqrt(mse_total))


Finalmente, exportamos el modelo a un archivo .pkl

In [ ]:
with open("./modelo_final.pkl", "wb") as file:
    pickle.dump(svm, file)

### 8. Transformación a problema de clasificación

En este último apartado se resolverá el punto final del enunciado:
1. se pide comprobar con el mejor modelo obtenido hasta el momento, si las predicciones para valores altos son peores que para valores bajos.
   - cuando la energía sea menor que el tercer cuantil, se considerará clase “baja”, y cuando sea mayor, clase “alta”.
2. se nos propone convertir el problema de regresión en uno de clasificación

In [ ]:
# Suponiendo que tienes el mejor modelo de regresión guardado en "best_regression_model"

# Realizar predicciones con el mejor modelo de regresión
predictions = svm.predict(X)

# Añadir las predicciones al DataFrame
wind_ava['predicted_energy'] = predictions

# Calcular el tercer cuartil de la energía
third_quantile = wind_ava['energy'].quantile(0.75)

# Calcular el error relativo para valores altos y bajos
err_rel_low = abs(wind_ava[wind_ava['energy'] <= third_quantile]['predicted_energy'] - wind_ava[wind_ava['energy'] <= third_quantile]['energy'])
err_rel_high = abs(wind_ava[wind_ava['energy'] > third_quantile]['predicted_energy'] - wind_ava[wind_ava['energy'] > third_quantile]['energy'])

print("Error relativo para valores bajos:", err_rel_low.mean())
print("Error relativo para valores altos:", err_rel_high.mean())


In [ ]:
# Transformación problema de regresión en clasificación:
# Crear una nueva columna para la variable objetivo con clases "baja" y "alta"
wind_ava_clases = wind_ava
wind_ava_clases['target_class'] = 'baja'
wind_ava_clases.loc[wind_ava['energy'] > third_quantile, 'target_class'] = 'alta'

# Dividir el DataFrame en características (X) y variable objetivo (y)
X_clases = wind_ava_clases[wind_ava.columns.difference(['target_class','datetime','predicted_energy'])]
Y_clases = wind_ava_clases['target_class']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train_clases = X_clases.loc[wind_ava.datetime < '2008-01-01']
X_test_clases = X_clases.loc[wind_ava.datetime >= '2008-01-01']
Y_train_clases = Y_clases.loc[wind_ava.datetime < '2008-01-01']
Y_test_clases = Y_clases.loc[wind_ava.datetime >= '2008-01-01']

# Entrenar un modelo de clasificación (por ejemplo, SVM)
clf = SVC()
clf.fit(X_train_clases, Y_train_clases)

# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test_clases)

# Calcular métricas de evaluación
print("Accuracy:", metrics.accuracy_score(Y_test_clases, y_pred))
print(metrics.classification_report(Y_test_clases, y_pred))


### 9. Uso de AI en nuestro trabajo
Para poder realizar este trabajo, se han usado, fundamentalmente, 2 AIs:
- **ChatGPT** : Para dudas de caracter general, ya sean explicación de los hiperparámetros, definición del proceso de evaluación, información adicional a cerca del proceso de realización de un EDA e interpretaciones generales de los resultados.
- **Github Copilot** : Ha sido la herramienta que más ha sido usada. Cuando se realizaban las evaluaciones, nossotros podíamos definir mediante un comentario, toda la creación del código en base a los requerimientos del comentario. Además, cuando nos saltaba un error en la ejecución, Copilot era nuestra guía para poder solventar los errores encontrados.

Cabe mencionar que la gran mayoría de información y conocimientos han sido extraídos de los videos publicados en Aulaglobal y las diapositivas del curso. Han sido unos recursos que nos han ido guiando bastante, sobre todo, en el proceso de evaluación. Las cuestiones que se alejaban más del temario enseñado han podido ser resultas gracias a las AIs comentadas previamente.